In [1]:
import pickle
from IPython.display import display
import pandas as pd

In [2]:
with open('Data_scrap.pkl', 'rb') as file:
    duomenu_sar = pickle.load(file)

Kiekvienai lentelei priskiriu unikalią vertę - datą. Jei reikia išsifiltruoju html scrappinimo likučius.

In [3]:
for el in duomenu_sar:
    data = el[0]
    for df in el[1:]:
        df['Data'] = data
        for col in df.columns:
            print(f"Column name: {repr(col)}")

        df.columns = df.columns.str.strip()
        df.columns = df.columns.str.replace('\u200b', '', regex=True)


Column name: 'ŽĮ\xa0\u200b\u200b'
Column name: '\u200b'
Column name: '2024 m. II pusm.  ct/kWh su PVM'
Column name: 'Data'
Column name: 'Tarifai, jų dedamosios'
Column name: 'Tarifai, jų dedamosios.1'
Column name: 'Mato vnt.'
Column name: '\u200b\u200b\u200b\u200bKaina\u200b'
Column name: '\u200b\u200b\u200b\u200bKaina\u200b.1'
Column name: 'Data'
Column name: 'ŽĮ \u200b\u200b\u200b\u200b\u200b'
Column name: 'ŽĮ \u200b\u200b\u200b\u200b\u200b.1'
Column name: '2024 m. I pusm. Eur/kWh su PVM'
Column name: 'Data'
Column name: 'Tarifai, jų dedamosios'
Column name: 'Tarifai, jų dedamosios.1'
Column name: 'Mato vnt.'
Column name: '\u200b\u200b\u200bKaina\u200b'
Column name: '\u200b\u200b\u200bKaina\u200b.1'
Column name: 'Data'
Column name: 'ŽĮ\xa0\u200b'
Column name: 'ŽĮ\xa0\u200b.1'
Column name: 'Mato vnt.'
Column name: '2024 m. I pusm.'
Column name: '2024 m. I pusm..1'
Column name: 'Data'
Column name: 'Tarifai, jų dedamosios'
Column name: 'Tarifai, jų dedamosios.1'
Column name: 'Mato vnt.'

In [4]:
def stulp_keitimas(df, pirmas='Tarifas', antras='Dedamoji', trecias='Kaina EUR/kWh su PVM'):
    df.rename(columns={df.columns[0]: pirmas, df.columns[1]: antras, df.columns[2]: trecias}, inplace=True)

def pavadinimu_keitimas(df, loc=1, pirmas='Pastovioji dedamoji. EUR/mėn.', antras='Viena laiko zona', trecias='Dieninė dedamoji', ketvirtas='Naktinė dedamoji'):
    naujos_vertes = [pirmas, antras, trecias, ketvirtas]
    if len(df) % 4 == 0:
        cycles = int(len(df)/4)
        for j in range(cycles):
            j *= len(naujos_vertes)
            for i, nauja_verte in enumerate(naujos_vertes):
                i += j
                df.iloc[i, loc] = nauja_verte


        
# verčių pasitikrinimui naudoju .to_list()

Deja lentelės yra tarpusavyje skirtingos, ir jose yra skirtingai pateikta informacija. Filtruoti automatiškai nepavyks.  
Duomenis susivedu rankiniu būdu, prieš tai atsispausdinamas kiekvieną lentelę atskirai.

***1. 2024 metai***

In [5]:
# filtruoju dataframe nr.1
duomenys1 = duomenu_sar[0][1]
lentele1_1 = duomenys1.iloc[3:7].copy()
duomenys2 = duomenu_sar[0][3]
lentele1_2 = duomenys2.iloc[3:7].copy()
lentele1_2.loc[:, 'Data'] = '2024-04-01'
stulp_keitimas(lentele1_1)
stulp_keitimas(lentele1_2)
lentele1_1.replace('', pd.NA, inplace=True)
lentele1_1.fillna('Planas "Namai"', inplace=True)
df_2024_1 = pd.concat([lentele1_1, lentele1_2], axis=0).reset_index(drop=True)
df_2024_1.iloc[0] = df_2024_1.iloc[0].str.replace('\u200b', '', regex=True) 
df_2024_1


,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
0,Planas „Namai“,Pastovioji dedamoji. EUR/mėn.,3,2024-07-01
1,"Planas ""Namai""",Viena laiko zona,0.181,2024-07-01
2,"Planas ""Namai""",Dvi laiko zonos,0.207,2024-07-01
3,"Planas ""Namai""",Naktinė dedamoji,0.123,2024-07-01
4,​​Planas „Namai“,Pastovioji dedamoji. EUR/mėn.,3,2024-04-01
5,​​Planas „Namai“,Viena laiko zona,0.198,2024-04-01
6,​​Planas „Namai“,Dvi laiko zonos,0.227,2024-04-01
7,​​Planas „Namai“,Naktinė dedamoji,0.138,2024-04-01


In [6]:
# filtruoju dataframe nr.2
duomenys3 = duomenu_sar[1][1]
lentele2 = duomenys3.iloc[5:9].copy()
df_2024_2 = lentele2.drop(columns=['Mato vnt.', '2024 m. I pusm..1'])
stulp_keitimas(df_2024_2)
df_2024_2

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
5,Planas „Namai“,Pastovioji dedamoji. EUR/mėn.,3.00,2024-03-31
6,Planas „Namai“,Viena laiko zona,0.203,2024-03-31
7,Planas „Namai“,Dvi laiko zonos.,0.232,2024-03-31
8,Planas „Namai“,dieninė dedamoji,0.232,2024-03-31


In [7]:
df_2024 = pd.concat([df_2024_1, df_2024_2])
pavadinimu_keitimas(df_2024)
df_2024

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
0,Planas „Namai“,Pastovioji dedamoji. EUR/mėn.,3,2024-07-01
1,"Planas ""Namai""",Viena laiko zona,0.181,2024-07-01
2,"Planas ""Namai""",Dieninė dedamoji,0.207,2024-07-01
3,"Planas ""Namai""",Naktinė dedamoji,0.123,2024-07-01
4,​​Planas „Namai“,Pastovioji dedamoji. EUR/mėn.,3,2024-04-01
5,​​Planas „Namai“,Viena laiko zona,0.198,2024-04-01
6,​​Planas „Namai“,Dieninė dedamoji,0.227,2024-04-01
7,​​Planas „Namai“,Naktinė dedamoji,0.138,2024-04-01
5,Planas „Namai“,Pastovioji dedamoji. EUR/mėn.,3.00,2024-03-31
6,Planas „Namai“,Viena laiko zona,0.203,2024-03-31


***2. 2023 metai***

In [8]:
# filtruoju dataframe nr.3
duomenys4 = duomenu_sar[2][1]
lentele3_1 = duomenys4.iloc[4:9].copy()
lentele3_2 = lentele3_1.copy()
df_2023_1 = lentele3_1.drop(columns=['Mato vnt.', '2023 m. kovo-birželio mėn. su tiekimo kainos kompensacija (su PVM)'])
df_2023_2 = lentele3_2.drop(columns=['Mato vnt.', '2023 m. II\xa0pusmetis (su PVM)'])
stulp_keitimas(df_2023_1)
stulp_keitimas(df_2023_2)
df_2023_2.loc[:, 'Data'] = '2023-07-01'
df_2023_1 = df_2023_1.drop(6)
df_2023_2 = df_2023_2.drop(6)
display(df_2023_1, df_2023_2)

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
4,​​​Planas „Namai“ ​ ​ ​ ​,Pastovioji dedamoji,3.00,2023-12-31
5,​​​Planas „Namai“ ​ ​ ​ ​,Vienos laiko zonos energijos dedamoji,0.211,2023-12-31
7,​​​Planas „Namai“ ​ ​ ​ ​,dieninė energijos dedamoji,0.243,2023-12-31
8,​​​Planas „Namai“ ​ ​ ​ ​,naktinė. šeštadienio ir sekmadienio energijos ...,0.149,2023-12-31


,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
4,​​​Planas „Namai“ ​ ​ ​ ​,Pastovioji dedamoji,3.00,2023-07-01
5,​​​Planas „Namai“ ​ ​ ​ ​,Vienos laiko zonos energijos dedamoji,0.265,2023-07-01
7,​​​Planas „Namai“ ​ ​ ​ ​,dieninė energijos dedamoji,0.280,2023-07-01
8,​​​Planas „Namai“ ​ ​ ​ ​,naktinė. šeštadienio ir sekmadienio energijos ...,0.183,2023-07-01


In [9]:
duomenys5 = duomenu_sar[4][1]
df_2023_3 = duomenys5.iloc[4:9]
df_2023_3 = df_2023_3.drop(columns=['Mato vnt.', '2023 m. I pusmečio be tiekimo kainos kompensacijos (su PVM)'])
stulp_keitimas(df_2023_3)
df_2023_3 = df_2023_3.drop(6)
df_2023 = pd.concat([df_2023_1, df_2023_2, df_2023_3])
pavadinimu_keitimas(df_2023)
df_2023

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
4,​​​Planas „Namai“ ​ ​ ​ ​,Pastovioji dedamoji. EUR/mėn.,3.00,2023-12-31
5,​​​Planas „Namai“ ​ ​ ​ ​,Viena laiko zona,0.211,2023-12-31
7,​​​Planas „Namai“ ​ ​ ​ ​,Dieninė dedamoji,0.243,2023-12-31
8,​​​Planas „Namai“ ​ ​ ​ ​,Naktinė dedamoji,0.149,2023-12-31
4,​​​Planas „Namai“ ​ ​ ​ ​,Pastovioji dedamoji. EUR/mėn.,3.00,2023-07-01
5,​​​Planas „Namai“ ​ ​ ​ ​,Viena laiko zona,0.265,2023-07-01
7,​​​Planas „Namai“ ​ ​ ​ ​,Dieninė dedamoji,0.280,2023-07-01
8,​​​Planas „Namai“ ​ ​ ​ ​,Naktinė dedamoji,0.183,2023-07-01
4,​​​Planas „Namai“ ​ ​ ​ ​,Pastovioji dedamoji. EUR/mėn.,3.00,2023-03-01
5,​​​Planas „Namai“ ​ ​ ​ ​,Viena laiko zona,0.280,2023-03-01


***3. 2022 metai***

In [10]:
metai_2022 = duomenu_sar[15][1]
metai_2022 = metai_2022.drop(metai_2022.columns[2], axis=1)
metai_2022 = metai_2022.drop([2, 3, 4, 7, 8, 9, 10, 11, 12, 13])
df_2022 = metai_2022
stulp_keitimas(df_2022)
pavadinimu_keitimas(df_2022)
df_2022


,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
0,Standartinis Planas,Pastovioji dedamoji. EUR/mėn.,3.000,2022-12-31
1,Standartinis Planas,Viena laiko zona,0.167,2022-12-31
5,Namai plius Planas,Dieninė dedamoji,0.189,2022-12-31
6,Namai plius Planas,Naktinė dedamoji,0.124,2022-12-31


***4. 2021 metai***

In [11]:
duomenys6 = duomenu_sar[5][1]
duomenys6 = duomenys6.drop([0, 1, 2, 3, 4, 5, 6, 9, 10, 13, 14, 15, 16, 17])
duomenys6 = duomenys6.drop(duomenys6.columns[[2, 3, 5, 6, 7, 8, 9, 10]], axis=1)
df_2021_1 = duomenys6
stulp_keitimas(df_2021_1)
df_2021_1.replace({'Kaina ct/kWh su PVM': {'-': 0}}, inplace=True)
df_2021_1

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
7,,Pastovioji dedamoji,-,2021-12-31
8,,Vienos laiko zonos energijos dedamoji,0.152,2021-12-31
11,,Dieninė energijos dedamoji,0.177,2021-12-31
12,,Naktinė. šeštadienio ir sekmadienio energijos ...,0.106,2021-12-31


In [12]:
duomenys7 = duomenu_sar[6][1]
duomenys7 = duomenys7.drop([0, 1, 2, 3, 6, 7, 10, 11, 12, 13, 14])
duomenys7 = duomenys7.drop(duomenys7.columns[[2, 3, 5, 6, 7, 8, 9, 10]], axis=1)
df_2021_2 = duomenys7
stulp_keitimas(df_2021_2)
df_2021_2.replace({'Kaina ct/kWh su PVM': {'-': 0}}, inplace=True)
df_2021_2

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
4,,Pastovioji dedamoji,-,2021-06-30
5,,Vienos laiko zonos energijos dedamoji,0.141,2021-06-30
8,,Dieninė energijos dedamoji,0.161,2021-06-30
9,,Naktinė. šeštadienio ir sekmadienio energijos ...,0.100,2021-06-30


In [13]:
df_2021 = pd.concat([df_2021_1, df_2021_2])
pavadinimu_keitimas(df_2021)
df_2021

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
7,,Pastovioji dedamoji. EUR/mėn.,-,2021-12-31
8,,Viena laiko zona,0.152,2021-12-31
11,,Dieninė dedamoji,0.177,2021-12-31
12,,Naktinė dedamoji,0.106,2021-12-31
4,,Pastovioji dedamoji. EUR/mėn.,-,2021-06-30
5,,Viena laiko zona,0.141,2021-06-30
8,,Dieninė dedamoji,0.161,2021-06-30
9,,Naktinė dedamoji,0.100,2021-06-30


***5. 2020 metai***

In [14]:
duomenys8 = duomenu_sar[7][1]
duomenys8 = duomenys8.drop([0, 1, 2, 3, 6, 7, 10, 11, 12, 13, 14])
duomenys8 = duomenys8.drop(duomenys8.columns[[2, 3, 5, 6, 7, 8, 9]], axis=1)
df_2020_1 = duomenys8
stulp_keitimas(df_2020_1)
df_2020_1.replace({'Kaina ct/kWh su PVM': {'-': 0}}, inplace=True)
df_2020_1

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
4,,Pastovioji dedamoji,-,2020-12-31
5,,Vienos laiko zonos energijos dedamoji,0.137,2020-12-31
8,,Dieninė energijos dedamoji,0.155,2020-12-31
9,,Naktinė. šeštadienio ir sekmadienio energijos ...,0.099,2020-12-31


In [15]:
duomenys9 = duomenu_sar[8][1]
duomenys9 = duomenys9.drop([0, 1, 2, 3, 6, 7, 10, 11, 12, 13 ,14])
duomenys9 = duomenys9.drop(duomenys9.columns[[2, 3, 5, 6, 7, 8, 9, 10]], axis=1)
df_2020_2 = duomenys9
stulp_keitimas(df_2020_2)
df_2020_2.replace({'Kaina ct/kWh su PVM': {'-': 0}}, inplace=True)
df_2020_2

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
4,,Pastovioji dedamoji,-,2020-06-30
5,,Vienos laiko zonos energijos dedamoji,0.149,2020-06-30
8,,Dieninė energijos dedamoji,0.168,2020-06-30
9,,Naktinė. šeštadienio ir sekmadienio energijos ...,0.107,2020-06-30


In [16]:
df_2020 = pd.concat([df_2020_1, df_2020_2])
pavadinimu_keitimas(df_2020)
df_2020

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
4,,Pastovioji dedamoji. EUR/mėn.,-,2020-12-31
5,,Viena laiko zona,0.137,2020-12-31
8,,Dieninė dedamoji,0.155,2020-12-31
9,,Naktinė dedamoji,0.099,2020-12-31
4,,Pastovioji dedamoji. EUR/mėn.,-,2020-06-30
5,,Viena laiko zona,0.149,2020-06-30
8,,Dieninė dedamoji,0.168,2020-06-30
9,,Naktinė dedamoji,0.107,2020-06-30


***6. 2019 metai***

In [17]:
duomenys10 = duomenu_sar[9][1]
duomenys10 = duomenys10.drop([0, 1, 2, 3, 4, 5, 6, 9, 10, 13, 14, 15, 16, 17])
duomenys10 = duomenys10.drop(duomenys10.columns[[2, 3, 5, 6, 7, 8, 9, 10]], axis=1)
df_2019 = duomenys10
stulp_keitimas(df_2019)
pavadinimu_keitimas(df_2019)
df_2019.replace({'Kaina ct/kWh su PVM': {'-': 0}}, inplace=True)
df_2019


,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
7,,Pastovioji dedamoji. EUR/mėn.,-,2019-12-31
8,,Viena laiko zona,0.130,2019-12-31
11,,Dieninė dedamoji,0.143,2019-12-31
12,,Naktinė dedamoji,0.100,2019-12-31


***7. 2018 metai***

In [18]:
duomenys11 = duomenu_sar[10][1]
duomenys11 = duomenys11.drop([0, 1, 2, 3, 4, 5, 6, 9, 10, 13, 14, 15, 16, 17])
duomenys11 = duomenys11.drop(duomenys11.columns[[2, 3, 5, 6, 7, 8, 9, 10]], axis=1)
df_2018 = duomenys11
stulp_keitimas(df_2018)
pavadinimu_keitimas(df_2018)
df_2018.replace({'Kaina ct/kWh su PVM': {'-': 0}}, inplace=True)
df_2018.iloc[:, 0] = ''
df_2018


,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
7,,Pastovioji dedamoji. EUR/mėn.,-,2018-12-31
8,,Viena laiko zona,0.113,2018-12-31
11,,Dieninė dedamoji,0.122,2018-12-31
12,,Naktinė dedamoji,0.091,2018-12-31


***8. 2017 metai***

In [19]:
duomenys12 = duomenu_sar[11][1]
duomenys12 = duomenys12.drop([0, 1, 2, 3, 4, 6, 8])
duomenys12 = duomenys12.drop(duomenys12.columns[[2, 3, 5, 6]], axis=1)
df_2017 = duomenys12
stulp_keitimas(df_2017)
pavadinimu_keitimas(df_2017)
df_2017.replace({'Kaina ct/kWh su PVM': {'': 0}}, inplace=True)
df_2017.iloc[:, 0] = ''
df_2017

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
5,,Pastovioji dedamoji. EUR/mėn.,,2017-12-31
7,,Viena laiko zona,0.114,2017-12-31
9,,Dieninė dedamoji,0.124,2017-12-31
10,,Naktinė dedamoji,0.091,2017-12-31


***6. 2016 metai***

In [20]:
duomenys13 = duomenu_sar[12][1]
duomenys13 = duomenys13.drop([0, 1, 2, 5])
duomenys13 = duomenys13.drop(duomenys13.columns[[2, 3, 5, 6]], axis=1)
df_2016_1 = duomenys13
stulp_keitimas(df_2016_1)
# pavadinimu_keitimas(df_2016_1)
df_2016_1.iloc[0, 2] = 0
df_2016_1.iloc[:, 0] = ''
df_2016_1

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
3,,6.1. Vienos laiko zonos tarifas:,0,2016-12-31
4,,vienos laiko zonos energijos dedamoji,0.120,2016-12-31
6,,dieninė energijos dedamoji,0.130,2016-12-31
7,,naktinė. šeštadienio ir sekmadienio energijos ...,0.096,2016-12-31


In [21]:
duomenys14 = duomenu_sar[13][1]
duomenys14 = duomenys14.drop([0, 1, 2, 5])
duomenys14 = duomenys14.drop(duomenys14.columns[[2, 3, 5, 6]], axis=1)
df_2016_2 = duomenys14
stulp_keitimas(df_2016_2)
# pavadinimu_keitimas(df_2016_2)
df_2016_2.iloc[0, 2] = 0
df_2016_2.iloc[:, 0] = ''
df_2016_2

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
3,,6.1. Vienos laiko zonos tarifas:,0,2016-06-30
4,,vienos laiko zonos energijos dedamoji,0.127,2016-06-30
6,,dieninė energijos dedamoji,0.139,2016-06-30
7,,naktinė. šeštadienio ir sekmadienio energijos ...,0.099,2016-06-30


In [22]:
df_2016 = pd.concat([df_2016_1, df_2016_2])
pavadinimu_keitimas(df_2016)
df_2016

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
3,,Pastovioji dedamoji. EUR/mėn.,0,2016-12-31
4,,Viena laiko zona,0.120,2016-12-31
6,,Dieninė dedamoji,0.130,2016-12-31
7,,Naktinė dedamoji,0.096,2016-12-31
3,,Pastovioji dedamoji. EUR/mėn.,0,2016-06-30
4,,Viena laiko zona,0.127,2016-06-30
6,,Dieninė dedamoji,0.139,2016-06-30
7,,Naktinė dedamoji,0.099,2016-06-30


***6. 2015 metai***

In [23]:
duomenys15 = duomenu_sar[14][1]
duomenys15 = duomenys15.drop([0, 1, 2, 3, 4, 5, 8])
duomenys15 = duomenys15.drop(duomenys15.columns[[2, 3, 5]], axis=1)
df_2015 = duomenys15
stulp_keitimas(df_2015)
pavadinimu_keitimas(df_2015)
df_2015.iloc[0, 2] = 0
df_2015.iloc[:, 0] = ''

df_2015.iloc[:, 2] = pd.to_numeric(df_2015.iloc[:, 2], errors='coerce') # Kitaip neatlieka matematinių veiksmų
df_2015.iloc[:, 2] = df_2015.iloc[:, 2] / 100

df_2015

,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
6,,Pastovioji dedamoji. EUR/mėn.,0.0,2015-12-31
7,,Viena laiko zona,0.105,2015-12-31
9,,Dieninė dedamoji,0.115,2015-12-31
10,,Naktinė dedamoji,0.083,2015-12-31


***Visų DF sulipdymas į vieną***

In [24]:
df_Elektros_kainos = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_2024])
df_Elektros_kainos


,Tarifas,Dedamoji,Kaina EUR/kWh su PVM,Data
6,,Pastovioji dedamoji. EUR/mėn.,0.0,2015-12-31
7,,Viena laiko zona,0.105,2015-12-31
9,,Dieninė dedamoji,0.115,2015-12-31
10,,Naktinė dedamoji,0.083,2015-12-31
3,,Pastovioji dedamoji. EUR/mėn.,0,2016-12-31
...,...,...,...,...
7,​​Planas „Namai“,Naktinė dedamoji,0.138,2024-04-01
5,Planas „Namai“,Pastovioji dedamoji. EUR/mėn.,3.00,2024-03-31
6,Planas „Namai“,Viena laiko zona,0.203,2024-03-31
7,Planas „Namai“,Dieninė dedamoji,0.232,2024-03-31


In [25]:
df_Elektros_kainos = df_Elektros_kainos.pivot(index='Data', columns='Dedamoji', values='Kaina EUR/kWh su PVM')
df_Elektros_kainos

Dedamoji,Dieninė dedamoji,Naktinė dedamoji,Pastovioji dedamoji. EUR/mėn.,Viena laiko zona
Data,,,,
2015-12-31,0.115,0.083,0.0,0.105
2016-06-30,0.139,0.099,0,0.127
2016-12-31,0.130,0.096,0,0.120
2017-12-31,0.124,0.091,,0.114
2018-12-31,0.122,0.091,-,0.113
2019-12-31,0.143,0.100,-,0.130
2020-06-30,0.168,0.107,-,0.149
2020-12-31,0.155,0.099,-,0.137
2021-06-30,0.161,0.100,-,0.141


In [26]:
for i in range(len(df_Elektros_kainos.iloc[:, 2])):
    if isinstance(df_Elektros_kainos.iloc[i, 2], str):
        if df_Elektros_kainos.iloc[i, 2].startswith('3'):
            df_Elektros_kainos.iloc[i, 2] = 3
        df_Elektros_kainos.iloc[i, 2] = 0
    elif df_Elektros_kainos.iloc[i, 2] == 0:
        df_Elektros_kainos.iloc[i, 2] = 0
    elif df_Elektros_kainos.iloc[i, 2] == 3:
        df_Elektros_kainos.iloc[i, 2] = 3
    else:
        df_Elektros_kainos.iloc[i, 2] = 0

    

In [27]:
df_Elektros_kainos.columns.name = None
df_Elektros_kainos.rename(columns={'index': 'Data'}, inplace=True)
df_Elektros_kainos

,Dieninė dedamoji,Naktinė dedamoji,Pastovioji dedamoji. EUR/mėn.,Viena laiko zona
Data,,,,
2015-12-31,0.115,0.083,0,0.105
2016-06-30,0.139,0.099,0,0.127
2016-12-31,0.130,0.096,0,0.120
2017-12-31,0.124,0.091,0,0.114
2018-12-31,0.122,0.091,0,0.113
2019-12-31,0.143,0.100,0,0.130
2020-06-30,0.168,0.107,0,0.149
2020-12-31,0.155,0.099,0,0.137
2021-06-30,0.161,0.100,0,0.141


***El. duomenys iš Litgrid Grafikų vaizdavimui vėliau***  

In [28]:
df_paruosimas = pd.read_excel(r'Databases\Ketvirtiniai.xlsx', dtype='unicode')
df_paruosimas = df_paruosimas.iloc[:15, :] # Trumpinu dataframe iki man reikalingų eilučių
df_paruosimas = df_paruosimas.filter(regex=r'^\d{4}$|^Unnamed')
df_paruosimas = df_paruosimas.drop([0, 6, 7]) # Filtruoju papildomas eilutes
df_paruosimas.fillna(0, inplace=True)
start = 2011

for i, col in enumerate(df_paruosimas.columns[1:]):
    df_paruosimas[col] = pd.to_numeric(df_paruosimas[col], errors='coerce').round(3)
    j = i % 4
    if j == 0:
        start += 1
    df_paruosimas.rename(columns={col: f'{start}-Q{j+1}'}, inplace=True)
df_paruosimas.rename(columns={'Unnamed: 0': 'Pavadinimas'}, inplace=True)
df_paruosimas.set_index('Pavadinimas', inplace=True)
df_paruosimas.index = df_paruosimas.index.str.strip()
df_paruosimas

,2012-Q1,2012-Q2,2012-Q3,2012-Q4,2013-Q1,2013-Q2,2013-Q3,2013-Q4,2014-Q1,2014-Q2,...,2021-Q3,2021-Q4,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4
Pavadinimas,,,,,,,,,,,,,,,,,,,,,
Elektros energijos gamyba (Neto),1.420,0.789,1.268,1.229,1.241,0.854,1.176,1.127,1.098,0.760,...,1.095,1.239,1.273,0.960,0.962,1.055,1.373,1.202,1.341,1.749
Šiluminės elektrinės,0.969,0.414,0.908,0.745,0.757,0.311,0.766,0.521,0.497,0.289,...,0.486,0.326,0.285,0.199,0.345,0.332,0.267,0.272,0.327,0.445
Kruonio HAE,0.131,0.113,0.129,0.141,0.126,0.158,0.119,0.139,0.135,0.143,...,0.132,0.218,0.170,0.111,0.105,0.158,0.146,0.105,0.125,0.143
Hidroelektrinės,0.127,0.122,0.064,0.108,0.133,0.178,0.083,0.123,0.144,0.109,...,0.067,0.104,0.173,0.128,0.077,0.079,0.185,0.113,0.054,0.095
Vėjo elektrinės,0.149,0.103,0.113,0.173,0.153,0.112,0.108,0.227,0.217,0.115,...,0.268,0.463,0.514,0.348,0.270,0.381,0.654,0.395,0.521,0.954
Kiti atsinaujinantys energijos ištekliai,0.040,0.042,0.054,0.062,0.071,0.096,0.101,0.116,0.105,0.104,...,0.142,0.128,0.131,0.174,0.165,0.105,0.121,0.317,0.314,0.108
"Elektrinės, kūrenamos biomase",0.040,0.042,0.053,0.061,0.068,0.057,0.061,0.077,0.073,0.056,...,0.022,0.054,0.044,0.028,0.024,0.044,0.042,0.023,0.022,0.036
"Elektrinės, kūrenamos biodujomis",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.038,0.037,0.037,0.033,0.024,0.040,0.035,0.025,0.019,0.024
Saulės energijos elektrinės,0.000,0.001,0.001,0.001,0.003,0.015,0.022,0.005,0.010,0.028,...,0.063,0.018,0.036,0.099,0.117,0.021,0.044,0.268,0.273,0.048


In [29]:
df_paruosimas['2024-Q1'] = 0.0
df_paruosimas['2024-Q2'] = 0.0


duomenys = {
    'Elektros energijos gamyba (Neto)': {'2024-Q1': 1.875, '2024-Q2': 1.864},
    'Šiluminės elektrinės': {'2024-Q1': 0.332, '2024-Q2': 0.252},
    'Vėjo elektrinės': {'2024-Q1': 0.977, '2024-Q2': 0.635},
    'Saulės energijos elektrinės': {'2024-Q1': 0.108, '2024-Q2': 0.592},
    'Hidroelektrinės': {'2024-Q1': 0.171, '2024-Q2': 0.133},
    'Kruonio HAE': {'2024-Q1': 0.101, '2024-Q2': 0.105},
    'Kiti atsinaujinantys energijos ištekliai': {'2024-Q1': 0.186, '2024-Q2': 0.147},
    'Komercinis sistemos balansas (-Importas/+eksportas)': {'2024-Q1': -1.038, '2024-Q2': -0.719}
}

# Update the DataFrame using the dictionary
for key, values in duomenys.items():
    df_paruosimas.loc[key, values.keys()] = values.values()

In [30]:
df_paruosimas

,2012-Q1,2012-Q2,2012-Q3,2012-Q4,2013-Q1,2013-Q2,2013-Q3,2013-Q4,2014-Q1,2014-Q2,...,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2
Pavadinimas,,,,,,,,,,,,,,,,,,,,,
Elektros energijos gamyba (Neto),1.420,0.789,1.268,1.229,1.241,0.854,1.176,1.127,1.098,0.760,...,1.273,0.960,0.962,1.055,1.373,1.202,1.341,1.749,1.875,1.864
Šiluminės elektrinės,0.969,0.414,0.908,0.745,0.757,0.311,0.766,0.521,0.497,0.289,...,0.285,0.199,0.345,0.332,0.267,0.272,0.327,0.445,0.332,0.252
Kruonio HAE,0.131,0.113,0.129,0.141,0.126,0.158,0.119,0.139,0.135,0.143,...,0.170,0.111,0.105,0.158,0.146,0.105,0.125,0.143,0.101,0.105
Hidroelektrinės,0.127,0.122,0.064,0.108,0.133,0.178,0.083,0.123,0.144,0.109,...,0.173,0.128,0.077,0.079,0.185,0.113,0.054,0.095,0.171,0.133
Vėjo elektrinės,0.149,0.103,0.113,0.173,0.153,0.112,0.108,0.227,0.217,0.115,...,0.514,0.348,0.270,0.381,0.654,0.395,0.521,0.954,0.977,0.635
Kiti atsinaujinantys energijos ištekliai,0.040,0.042,0.054,0.062,0.071,0.096,0.101,0.116,0.105,0.104,...,0.131,0.174,0.165,0.105,0.121,0.317,0.314,0.108,0.186,0.147
"Elektrinės, kūrenamos biomase",0.040,0.042,0.053,0.061,0.068,0.057,0.061,0.077,0.073,0.056,...,0.044,0.028,0.024,0.044,0.042,0.023,0.022,0.036,0.000,0.000
"Elektrinės, kūrenamos biodujomis",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.037,0.033,0.024,0.040,0.035,0.025,0.019,0.024,0.000,0.000
Saulės energijos elektrinės,0.000,0.001,0.001,0.001,0.003,0.015,0.022,0.005,0.010,0.028,...,0.036,0.099,0.117,0.021,0.044,0.268,0.273,0.048,0.108,0.592


In [31]:
with open('Formatuota_lentele.pkl', 'wb') as file:
    pickle.dump(df_Elektros_kainos, file)
with open('Litgrid_lentele.pkl', 'wb') as file:
    pickle.dump(df_paruosimas, file)